Criar uma ferramenta online (protótipo) de visualização de dados
- Quais as perguntas a serem respondidas?
- Quais análises são necessárias?
- Quais análises (extras) são possíveis?
- Ferramentas a utilizar:
    - Python (Notebook)
    - HTML/Javascript

In [1]:
# pip install pandas
# pip install plotly
# pip install os
# pip install datetime

In [2]:
import pandas as pd
import numpy as np
import plotly.express as px
import os
from datetime import datetime

In [3]:
df = {}

for archive in os.listdir('dados/dados_fornec'):
    data = pd.read_csv(f'dados/dados_fornec/{archive}')
    df[f'{archive.split(".")[0]}'] = pd.DataFrame(data)

### Agências

In [4]:
df['agencias'].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   cod_agencia    10 non-null     int64 
 1   nome           10 non-null     object
 2   endereco       10 non-null     object
 3   cidade         10 non-null     object
 4   uf             10 non-null     object
 5   data_abertura  10 non-null     object
 6   tipo_agencia   10 non-null     object
dtypes: int64(1), object(6)
memory usage: 688.0+ bytes


Perguntas:
- Quantas agências foram abertas por ano?

### Clientes

In [5]:
df['clientes'].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 998 entries, 0 to 997
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   cod_cliente      998 non-null    int64 
 1   primeiro_nome    998 non-null    object
 2   ultimo_nome      998 non-null    object
 3   email            998 non-null    object
 4   tipo_cliente     998 non-null    object
 5   data_inclusao    998 non-null    object
 6   cpfcnpj          998 non-null    object
 7   data_nascimento  998 non-null    object
 8   endereco         998 non-null    object
 9   cep              998 non-null    object
dtypes: int64(1), object(9)
memory usage: 78.1+ KB


Perguntas:
- Qual a faixa etária dos clientes?

### Colaborador-Agência

In [6]:
df['colaborador_agencia'].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 2 columns):
 #   Column           Non-Null Count  Dtype
---  ------           --------------  -----
 0   cod_colaborador  100 non-null    int64
 1   cod_agencia      100 non-null    int64
dtypes: int64(2)
memory usage: 1.7 KB


Perguntas:
- Quantos coleboradores trabalham em cada agência?

### Colaboradores

In [7]:
df['colaboradores'].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   cod_colaborador  100 non-null    int64 
 1   primeiro_nome    100 non-null    object
 2   ultimo_nome      100 non-null    object
 3   email            100 non-null    object
 4   cpf              100 non-null    object
 5   data_nascimento  100 non-null    object
 6   endereco         100 non-null    object
 7   cep              100 non-null    object
dtypes: int64(1), object(7)
memory usage: 6.4+ KB


Perguntas:
- Qual a faixa etária dos colaboradores?
- Qual a distribuição dos colaboradores por gênero?

### Contas

In [8]:
df['contas'].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 999 entries, 0 to 998
Data columns (total 9 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   num_conta               999 non-null    int64  
 1   cod_cliente             999 non-null    int64  
 2   cod_agencia             999 non-null    int64  
 3   cod_colaborador         999 non-null    int64  
 4   tipo_conta              999 non-null    object 
 5   data_abertura           999 non-null    object 
 6   saldo_total             999 non-null    float64
 7   saldo_disponivel        999 non-null    float64
 8   data_ultimo_lancamento  999 non-null    object 
dtypes: float64(2), int64(4), object(3)
memory usage: 70.4+ KB


Perguntas:
- Quantas contas cada agência possui?
- Quantas contas cada colaborador abriu/gerencia?
- Quantas contas foram abertas por ano?
- Quantas contas ainda estão ativas?
- Qual o perfil dos clientes?
- Quais colaboradores gerenciam mais contas?
- Quais colaboradores gerenciam menos contas?

In [9]:
contas_por_agencia = df['contas']['cod_agencia'].value_counts() # estudar quais agências tem maior movimentação
contas_por_colaborador = df['contas']['cod_colaborador'].value_counts() # estudar a eficiência de cada colaborador e relacionar a sua eficiência à movimentação da agência

In [10]:
fig = px.box(pd.DataFrame(contas_por_colaborador), points="all")

fig.show()

Intervalos:
- Q1: 1 a 4
- Q2: 4 a 6
- Q3: 6 a 9
- Q4: 9 a 13

In [11]:
fig = px.box(df['contas']['saldo_disponivel'], points="all")

fig.show()

Intervalos:
- Q1: 0 a 920 (Baixo)
- Q2: 920 a 11326 (Médio)
- Q3: 11326 a 33995 (Alto)
- Q4: 33995 a 83480 (Muito Alto)
- Acima de 83480 (Premium)

In [12]:
ano_abertura = []
ano_ultimo_lanc = []

for i in df['contas'].index:
    ano_abertura.append(int(datetime.strptime(df['contas']['data_abertura'][i].split()[0], "%Y-%m-%d").strftime("%Y")))
    ano_ultimo_lanc.append(int(datetime.strptime(df['contas']['data_ultimo_lancamento'][i].split()[0], "%Y-%m-%d").strftime("%Y")))

df['contas']['ano_abertura'] = ano_abertura
df['contas']['ano_ultimo_lanc'] = ano_ultimo_lanc

ano_abert_df = pd.DataFrame(df['contas']['ano_abertura'].value_counts())
ano_abert_df = ano_abert_df.reset_index()
ano_abert_df = ano_abert_df.rename(columns={'index': 'ano_abert', 'ano_abertura':'valores'})
ano_abert_df = ano_abert_df.sort_values(by='ano_abert')

px.line(ano_abert_df, x='ano_abert', y='valores')

In [13]:
tempo_inativo = []
for i in df['contas'].index:
    tempo_inativo.append(df['contas']['ano_ultimo_lanc'][i] - df['contas']['ano_abertura'][i])

df['contas']['t_inat'] = tempo_inativo

In [14]:
fig = px.box(df['contas']['t_inat'], points="all")

fig.show()

Intervalos:
- Q1: 0 a 1
- Q2: 1 a 3
- Q3: 3 a 5
- Q4: 5 a 11

### Propostas de Crédito

In [15]:
df['propostas_credito'].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   cod_proposta           2000 non-null   int64  
 1   cod_cliente            2000 non-null   int64  
 2   cod_colaborador        2000 non-null   int64  
 3   data_entrada_proposta  2000 non-null   object 
 4   taxa_juros_mensal      2000 non-null   float64
 5   valor_proposta         2000 non-null   float64
 6   valor_financiamento    2000 non-null   float64
 7   valor_entrada          2000 non-null   float64
 8   valor_prestacao        2000 non-null   float64
 9   quantidade_parcelas    2000 non-null   int64  
 10  carencia               2000 non-null   int64  
 11  status_proposta        2000 non-null   object 
dtypes: float64(5), int64(5), object(2)
memory usage: 187.6+ KB


In [16]:
df['propostas_credito']

,cod_proposta,cod_cliente,cod_colaborador,data_entrada_proposta,taxa_juros_mensal,valor_proposta,valor_financiamento,valor_entrada,valor_prestacao,quantidade_parcelas,carencia,status_proposta
0,116,338,1,2014-05-30 18:43:12 UTC,0.0194,36199.950355,50032.03,13832.079645,1045.219825,58,0,Enviada
1,715,45,1,2021-06-17 13:09:32 UTC,0.0131,12897.538285,19848.55,6951.011715,232.124685,100,1,Enviada
2,755,494,1,2021-08-10 04:24:02 UTC,0.0203,122236.312551,187154.36,64918.047449,3673.518951,56,0,Enviada
3,953,381,1,2011-02-23 07:57:58 UTC,0.0167,47784.649757,59530.61,11745.960243,955.041560,109,3,Enviada
4,1046,176,1,2010-11-15 16:03:28 UTC,0.0211,178393.038243,214294.19,35901.151757,10219.659415,22,4,Enviada
...,...,...,...,...,...,...,...,...,...,...,...,...
1995,1758,176,99,2010-01-10 08:12:47 UTC,0.0133,96404.111608,184043.36,87639.248392,1673.386037,110,6,Validação documentos
1996,582,241,100,2019-02-25 05:03:22 UTC,0.0151,61292.834593,82684.29,21391.455407,2219.574245,36,2,Validação documentos
1997,970,106,100,2011-06-05 04:13:16 UTC,0.0225,100371.681790,152272.50,51900.818210,3482.000980,47,2,Validação documentos
1998,1361,814,100,2020-07-25 20:50:30 UTC,0.0227,45431.892941,74490.36,29058.467059,1326.031489,67,5,Validação documentos


In [17]:
df['propostas_credito']['cod_colaborador'].value_counts()

18    34
99    33
73    33
27    31
37    30
      ..
35    11
9     11
8     10
57    10
50     8
Name: cod_colaborador, Length: 100, dtype: int64

In [18]:
df['propostas_credito']['status_proposta'].value_counts()

Enviada                 527
Aprovada                514
Validação documentos    491
Em análise              468
Name: status_proposta, dtype: int64

### Transações

In [19]:
df['transacoes'].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71999 entries, 0 to 71998
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   cod_transacao    71999 non-null  int64  
 1   num_conta        71999 non-null  int64  
 2   data_transacao   71999 non-null  object 
 3   nome_transacao   71999 non-null  object 
 4   valor_transacao  71999 non-null  float64
dtypes: float64(1), int64(2), object(2)
memory usage: 2.7+ MB


In [23]:
mes = []
ano = []

for i in range(len(df['transacoes']['data_transacao'])):
    mes.append(int(datetime.strptime(df['transacoes']['data_transacao'][i].split()[0], "%Y-%m-%d").strftime("%m")))
    ano.append(int(datetime.strptime(df['transacoes']['data_transacao'][i].split()[0], "%Y-%m-%d").strftime("%Y")))

df['transacoes']['ano'] = ano
df['transacoes']['mes'] = mes

# uma análise inicial dos dados permitiu identificar que os dados do ano 2023 estão incompletos, havendo informações referentes apenas ao primeiro trimestre, logo este ano será removido do conjunto de dados para as análises seguintes

df['transacoes'] = df['transacoes'].drop(df['transacoes'][df['transacoes']['ano'] == 2023].index)

In [24]:
transac_por_conta = df['transacoes']['num_conta'].value_counts()

agencias = {}

for i in df['agencias']['cod_agencia'].unique():
    df_local = pd.DataFrame(df['contas'][df['contas']['cod_agencia'] == i]['cod_cliente'])
    df_local['num_trans'] = df_local['cod_cliente'].map(transac_por_conta)
    agencias[i] = df_local['num_trans'].sum(axis=0)

A agência com maior quantidade de transações é a Agência Digital, o que aponta para o maior investimento

In [26]:
df['agencias']['num_trans'] = df['agencias']['cod_agencia'].map(agencias)

fig = px.bar(df['agencias'], x='nome', y='num_trans', color='tipo_agencia')

fig.show()

In [27]:
tipo_transf = {}

anos = sorted(df['transacoes']['ano'].unique(), key=int)

for a in anos:
    tipo_transf[a] = df['transacoes'][df['transacoes']['ano'] == a]['nome_transacao'].value_counts()

tipo_transf_df = pd.DataFrame(tipo_transf).reset_index()

tipo_transf_df = tipo_transf_df.rename(columns={'index': 'transacao'})

dados_long_1 = tipo_transf_df.melt(id_vars=["transacao"], var_name="ano", value_name="valor")

fig = px.bar(dados_long_1, x='transacao', y='valor', color='ano')

fig.show()

### Entrega 1

Nem todos os envolvidos estão convictos com a ideia de uma frente de BIs no BanVic. Como podemos convencer Camila Diniz, diretora comercial,  que as nossas soluções serão úteis para a empresa? Apresente sua análise exploratória realizada a partir dos dados apresentados e cite quais perguntas de negócio podemos responder a fim de mostrar o valor que dados possuem para a empresa, levando em consideração o desafio apresentado.

Descrição dos dados:
- Agências: informações características sobre as agências (e.g. nome, endereço). 10 entradas (relacional);
- Clientes: informações características sobre os clientes (e.g. nome, endereço, cpf). 998 entradas;
- Colaboradores: informações sobre qual agência cada colaborador pertence. 100 entradas (relacional);
- Colaborador_agência: relação entre colaborador e agência. 100 entradas;
- Contas: informações sobre as contas abertas neste banco. 999 entradas;
- Propostas: informações sobre as propostas de crédito feitas para cada cliente. 2000 entradas;
- Transações: informações sobre a quantidade de transações realizadas pelos clientes. 71999 entradas.

Perguntas de negócio:
- Quais os tipos de clientes que possuem contas no banco (físico ou jurídico)?
    - KPI: Número de clientes físicos/jurídicos;
- Considerando os tipos de agência (digital e física), qual tipo tem mais clientes?
    - KPI: número de clientes por tipo de agência;
- Baseado na localização, qual das agências tem mais clientes e por quê?
    - KPI: Número de clientes por agência;
    - Visualização: gráfico de barras por tipo
- Qual ano teve mais abertura de contas?
    - KPI: número de contas abertas por ano
- Qual o tipo de transação mais frequente?
    - KPI: número de transações por tipo
    - Visualização: gráfico de barras por tipo
- Qual a frequência de transações?
    - KPI: Número de transações por mês;
    - KPI: Número de transações por trimestre;
    - KPI: Número de transações por ano;
- Baseado no saldo em conta, quais os tipos de clientes que possuem conta no banco?
- Média do valor das transações;
- Média de transações por dia;
- Quantas propostas estão ativas?
- Quantas contas foram abertas por cada colaborador?
- Clientes que não fazem transações a muito tempo;
<!-- - Número de transações aprovadas e não-aprovadas; -->

### Entrega 2

Quais serão os indicadores ou análises utilizadas para responder às perguntas de negócio citadas anteriormente?

### Entrega 3

Uma dimensão de datas bem construída viabiliza a elaboração de análises mais robustas com relação ao tempo. O BanVic quer começar seu Data Warehouse (DW) com uma dim_dates que atenda seus requisitos de análise. Utilizando uma dimensão de datas, responda às duas perguntas a seguir. Por fim, proponha outras duas análises utilizando a dim_dates que possam ter valor de negócio para o BanVic.

- Qual trimestre tem, em média, mais transações aprovadas e qual tem, também em média,  maior volume movimentado?

Dado que não há no banco de dados um indicativo de quais transações foram aprovadas ou rejeitadas, assume-se que todas as transações contidas na BD foram aprovadas.

In [28]:
num_transf = {}

meses = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
anos = sorted(df['transacoes']['ano'].unique(), key=int)

for a in anos:
    count = df['transacoes'][df['transacoes']['ano'] == a]['mes'].value_counts()

    for m in meses:
        if m not in list(count.keys()):
            count[m] = 0

    num_transf[a] = {1: count[1] + count[2] + count[3], 
                     2: count[4] + count[5] + count[6], 
                     3: count[7] + count[8] + count[9], 
                     4: count[10] + count[11] + count[12]}

num_transf_df = pd.DataFrame(num_transf)
num_transf_df = num_transf_df.reset_index()
num_transf_df = num_transf_df.rename(columns={'index': 'Trimestre'})

In [29]:
vol_transf = {}

anos = sorted(df['transacoes']['ano'].unique(), key=int)

for a in anos:
    valor_prim_tri = 0
    valor_seg_tri = 0
    valor_ter_tri = 0
    valor_qua_tri = 0

    meses = sorted(df['transacoes'][df['transacoes']['ano'] == a]['mes'].unique(), key=int)

    df_new = df['transacoes'][df['transacoes']['ano'] == a]

    for m in meses:
        if m in [1, 2, 3]:
            valor_prim_tri += sum(abs(df_new[df_new['mes'] == m]['valor_transacao']))
        elif m in [4, 5, 6]:
            valor_seg_tri += sum(abs(df_new[df_new['mes'] == m]['valor_transacao']))
        elif m in [7, 8, 9]:
            valor_ter_tri += sum(abs(df_new[df_new['mes'] == m]['valor_transacao']))
        elif m in [10, 11, 12]:
            valor_qua_tri += sum(abs(df_new[df_new['mes'] == m]['valor_transacao']))

    vol_transf[a] = {1: valor_prim_tri, 2: valor_seg_tri, 3: valor_ter_tri, 4: valor_qua_tri}

vol_transf_df = pd.DataFrame(vol_transf)
vol_transf_df = vol_transf_df.reset_index()
vol_transf_df = vol_transf_df.rename(columns={'index': 'Trimestre'})

In [30]:
media = {'Trimestre': [1, 2, 3, 4], 'Número de Transferências': [], 'Volume Transferido': []}

for i in range(4):
    media['Número de Transferências'].append(np.mean(list(num_transf_df[num_transf_df['Trimestre'] == i + 1].loc[:, 2010:2022].values)))
    media['Volume Transferido'].append(np.mean(list(vol_transf_df[vol_transf_df['Trimestre'] == i + 1].loc[:, 2010:2022].values)))

In [31]:
pd.DataFrame(media)

,Trimestre,Número de Transferências,Volume Transferido
0,1,788.769231,7.110626e+05
1,2,845.461538,6.644091e+05
2,3,937.692308,9.016052e+05
3,4,2923.461538,2.163221e+06


- Um analista sugeriu que, meses que contém R no seu nome (Ex: janeiRo), tem um padrão de quantidade de transações diferentes dos meses que não apresentam R no nome (Ex: junho). Apresente suas análises validando ou não essa afirmação.

In [32]:
contagem_2 = {}

meses = {1: 'Jan', 2: 'Fev', 3: 'Mar', 4: 'Abr', 5: 'Mai', 6: 'Jun', 7: 'Jul', 8: 'Ago', 9: 'Set', 10: 'Out', 11: 'Nov', 12: 'Dez'}
anos = sorted(df['transacoes']['ano'].unique(), key=int)

for a in anos:
    contagem_2[a] = df['transacoes'][df['transacoes']['ano'] == a]['mes'].value_counts()
    contagem_2[a].columns = ['count']

data_3 = pd.DataFrame(contagem_2)
data_3 = data_3.reset_index()
data_3 = data_3.rename(columns={'index': 'mes'})
data_3 = data_3.fillna(0)

data_3['mes_nome'] = data_3['mes'].map(meses)

cont = {'mes': list(data_3['mes_nome']), 'valores': list(data_3.sum(axis=1)), 'com_r_flag': [True, True, True, True, False, False, False, False, True, True, True, True]}

cont_df = pd.DataFrame(cont)

fig = px.bar(cont_df, x='mes', y='valores', color='com_r_flag', labels={'mes': 'Mês', 'valores': 'Valores', 'com_r_flag': 'Meses com R'})

fig.update_layout(barmode='stack', xaxis={'categoryorder': 'array', 'categoryarray': list(meses.values())})

fig.show()

C:\Users\Adel\AppData\Local\Temp\ipykernel_12584\2254316994.py:17: FutureWarning:

Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.



In [33]:
data_3 = data_3.drop(columns='mes')

dados_long_2 = data_3.melt(id_vars=["mes_nome"], var_name="Ano", value_name="Valor")

fig = px.bar(dados_long_2, x='mes_nome', y='Valor', labels={'mes_nome': 'Mês'}, color='Ano')

fig.show()

In [34]:
data_3 = data_3.drop(columns=2022)

dados_long_2 = data_3.melt(id_vars=["mes_nome"], var_name="Ano", value_name="Valor")

fig = px.bar(dados_long_2, x='mes_nome', y='Valor', labels={'mes_nome': 'Mês'}, color='Ano')

fig.show()

### Entrega 4

André Tech, o especialista em tecnologia, solicitou que você buscasse fontes de dados públicas que poderiam ser utilizadas para enriquecer a base de dados do BanVic e ampliar as possibilidades de análise, considerando principalmente a necessidade atual apresentada. Pergunta prática e teórica.
- Traga uma proposta de possíveis dados públicos a serem utilizados no Data Warehouse apresentando o valor que eles podem trazer para o desafio apresentado. 

- Um exemplo de dado externo é o Índice Nacional de Preços ao Consumidor Amplo. Considerando o IPCA no Brasil para o período apresentado na base de dados da BanVic, avalie se existe alguma relação do índice de preços com o volume de transações e/ou com o valor transacionado no período. Apresente suas análises e justifique sua opinião.

In [35]:
fig = px.line(dados_long_2, x='mes_nome', y='Valor', labels={'mes_nome': 'Mês'}, color='Ano')

fig.show()

In [36]:
ipca_data = pd.read_csv('dados/dados_auxiliar/dados_ipca.csv')
ipca_df = pd.DataFrame(ipca_data)

ipca_df = ipca_df[29:41] # considerando os anos de 2010 a 2021

ipca_df = ipca_df.set_index('Ano').T.reset_index()

ipca_df.columns.name = None

ipca_df = ipca_df.rename(columns={'index': 'Mes'})
dados_long_3 = ipca_df.melt(id_vars=["Mes"], var_name="Ano", value_name="Valor")

fig = px.line(dados_long_3, x="Mes", y="Valor", color="Ano",
              title="Evolução Trimestral por Ano")

fig.show()

In [37]:
data_4 = df['transacoes'].loc[:, 'valor_transacao':'mes']

valor_trans = {'Mes' : ['Jan','Fev','Mar','Abr','Mai','Jun','Jul','Ago','Set','Out','Nov','Dez']}

meses = [1,2,3,4,5,6,7,8,9,10,11,12]
anos = sorted(df['transacoes']['ano'].unique(), key=int)

for a in anos:
    valor_trans[a] = []
    for m in meses:
        valor_trans[a].append(sum(data_4[(data_4['ano'] == a) & (data_4['mes'] == m)]['valor_transacao']))

df_transf = pd.DataFrame(valor_trans) # dataframe de valores das transferencias (2010 a 2021)

df_transf = df_transf.loc[:, 'Mes': 2021]

dados_long_4 = df_transf.melt(id_vars=["Mes"], var_name="Ano", value_name="Valor")

fig = px.line(dados_long_4, x="Mes", y="Valor", color="Ano",
              title="Evolução Trimestral por Ano")

fig.show()

In [38]:
labels = ['IPCA', 'Valor Transferido', 'Número de Transferências']
ipca_anual = ipca_df.loc[:, 2010:2021].sum(axis=0) # ipca anual
valor_transf_ano = df_transf.loc[:, 2010:2021].sum(axis=0) # valores transferidos anualmente
num_transf_anual = data_3.loc[:, 2010:2021].sum(axis=0) # número de transferências por ano

df_un = pd.DataFrame([ipca_anual, valor_transf_ano, num_transf_anual])
df_un.insert(0, 'ind', labels)

df_un = df_un.set_index('ind').T.reset_index()

df_un.columns.name = None

df_un = df_un.rename(columns={'index': 'Ano'})

Correlação de Pearson: https://www.blog.psicometriaonline.com.br/o-que-e-correlacao-de-pearson/

In [39]:
matriz_corr = df_un[['IPCA', 'Valor Transferido', 'Número de Transferências']].corr()

fig = px.imshow(matriz_corr,     text_auto=True,  # Exibe os valores dentro do heatmap
    color_continuous_scale="RdBu_r",  # Mapa de cores (azul-vermelho)
    labels=dict(color="Correlação")  # Nome da legenda de cores)
)

# Títulos e layout
fig.update_layout(
    title="Matriz de Correlação entre IPCA e Indicadores do BanVic",
    xaxis_title="Variáveis",
    yaxis_title="Variáveis"
)

fig.show()

In [ ]:
dados_long_5 = df_un.melt(id_vars=['Ano'], var_name='Indicador', value_name='Valor')

fig = px.line(dados_long_5, x="Ano", y="Valor", color="Indicador", markers=True,
              title="Evolução Trimestral por Ano")

fig.show()

Normalização com Z-score: https://www.investopedia.com/terms/z/zscore.asp

In [40]:
from scipy.stats import zscore

df_un['Valor Transferido Normalizado'] = zscore(df_un['Valor Transferido'])
df_un['Número de Transferências Normalizado'] = zscore(df_un['Número de Transferências'])
df_un['IPCA Normalizado'] = zscore(df_un['IPCA'])

df_un_edit = df_un[['Ano', 'Valor Transferido Normalizado', 'Número de Transferências Normalizado', 'IPCA Normalizado']]

dados_long_6 = df_un_edit.melt(id_vars=['Ano'], var_name='Score', value_name='Valor')

fig = px.line(dados_long_6, x="Ano", y="Valor", color="Score",
              title="Valor Transferido, Número de Transferências e IPCA Normalizados por Z-score", 
              labels={'valor_transf_zscore': 'Valor Transferido Normalizado', 'num_transf_zscore': 'Número de Transferências Normalizado', 'ipca_zscore': 'IPCA Normalizado', 'Score': 'Legenda'}
              )

fig.show()

- Quais decisões poderiam ser facilitadas com o uso desses novos dados? Pergunta de nível teórico. O seu projeto final não necessariamente precisa trazer esses dados externos.